In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('D:\Data Science\Bioenergy proj\Intermediate\Forestry_Biomass_DT.xlsx')
df.columns = df.iloc[0]
df.drop(0,axis=0, inplace=True)

In [3]:
df

,Sub-County,Forestland (Ha),Woodland (Ha),Shrubland (Ha),Forest dynamic between 2001-2021,Change period,Extent of change (Ha),2019_Household,Household’s access to land (ha),Average fraction of a smallholder farm under trees,Average proportion/percentage of smallholder farms under trees)
1,Mbooni,2750,2939,411,Loss,20,446,44003,1.58,0.27,0.27
2,Kilome,1192,148,456,Loss,20,246,28132,1.58,0.27,0.01
3,Kaiti,3628,945,0,Loss,20,321,28455,1.58,0.45,0.87
4,Makueni,1265,1293,2314,Loss,20,468,50521,1.58,0.1,0.13
5,Kibwezi West,815,442,1069,Loss,20,473,50889,1.58,0.25,0.24
6,Kibwezi East,93,57,8459,Loss,20,138,34267,1.58,0.25,0.05


In [4]:
Lands = df.iloc[:,0:4]
for i in Lands.iloc[:,1:].columns:
    globals()[f"{i[:-5]}_sum"] = Lands[i].sum()
    Lands[i+"percent"]= Lands[i]/globals()[f"{i[:-5]}_sum"]

In [5]:
Lands

,Sub-County,Forestland (Ha),Woodland (Ha),Shrubland (Ha),Forestland (Ha)percent,Woodland (Ha)percent,Shrubland (Ha)percent
1,Mbooni,2750,2939,411,0.282254,0.504636,0.032339
2,Kilome,1192,148,456,0.122344,0.025412,0.03588
3,Kaiti,3628,945,0,0.37237,0.16226,0.0
4,Makueni,1265,1293,2314,0.129837,0.222012,0.182076
5,Kibwezi West,815,442,1069,0.08365,0.075893,0.084114
6,Kibwezi East,93,57,8459,0.009545,0.009787,0.665591


In [6]:
def percent_lands(data:pd.DataFrame) ->pd.DataFrame:
    """Calculate the percentage of land cover types in each pixel for a given dataset"""
    Lands1 = data.iloc[:,0:4]
    for i in Lands1.iloc[:,1:].columns:
        globals()[f"{i[:-5]}_sum"] = Lands1[i].sum()
        Lands1[i+"percent"]= Lands[i]/globals()[f"{i[:-5]}_sum"]
    return Lands1
    

In [7]:
percent_lands(df)

,Sub-County,Forestland (Ha),Woodland (Ha),Shrubland (Ha),Forestland (Ha)percent,Woodland (Ha)percent,Shrubland (Ha)percent
1,Mbooni,2750,2939,411,0.282254,0.504636,0.032339
2,Kilome,1192,148,456,0.122344,0.025412,0.03588
3,Kaiti,3628,945,0,0.37237,0.16226,0.0
4,Makueni,1265,1293,2314,0.129837,0.222012,0.182076
5,Kibwezi West,815,442,1069,0.08365,0.075893,0.084114
6,Kibwezi East,93,57,8459,0.009545,0.009787,0.665591


In [8]:
def Annual_change_kha(data:pd.DataFrame) ->pd.DataFrame:
    """
    Here we are Accounting for effect of sub-division of group ranches and
    subsequent clearance of woody biomass. Source: Global Forest Watch

    Annual rate of change (loss) of tree cover (outside cropland)  
    """
    changes = data[["Sub-County","Forest dynamic between 2001-2021",	"Change period","Extent of change (Ha)"]]
    changes['Annual rate of change (kha/yr)'] =(changes['Extent of change (Ha)']/1000)/ changes['Change period']
    changes['Annual rate of change (ha/yr)'] = changes['Annual rate of change (kha/yr)']*1000
    return changes


In [9]:
Annual_change_kha(df)

,Sub-County,Forest dynamic between 2001-2021,Change period,Extent of change (Ha),Annual rate of change (kha/yr),Annual rate of change (ha/yr)
1,Mbooni,Loss,20,446,0.0223,22.3
2,Kilome,Loss,20,246,0.0123,12.3
3,Kaiti,Loss,20,321,0.01605,16.05
4,Makueni,Loss,20,468,0.0234,23.4
5,Kibwezi West,Loss,20,473,0.02365,23.65
6,Kibwezi East,Loss,20,138,0.0069,6.9


In [10]:
def Households_accounting(data: pd.DataFrame)->pd.DataFrame:
    """
    Here we are Accounting for increase in household number and expected growth of farm-forestry and getting the projected PNH

    """
    stats = data[['Sub-County','2019_Household']]
    stats = stats.reset_index().drop('index', axis=1)
    dict1 = {}
    for i in range(2020,2033,1):
        dict1[f"{i}_Household"] = []
    for houses in stats['2019_Household']:
        x = houses
        for households in dict1:
            x*=1.0262
            dict1[households].append(x)
    dict1
    return pd.concat([stats,pd.DataFrame(dict1)], axis=1)

In [11]:
Households_accounting(df)

,Sub-County,2019_Household,2020_Household,2021_Household,2022_Household,2023_Household,2024_Household,2025_Household,2026_Household,2027_Household,2028_Household,2029_Household,2030_Household,2031_Household,2032_Household
0,Mbooni,44003,45155.8786,46338.962619,47553.043440,48798.933178,50077.465227,51389.494816,52735.899580,54117.580149,55535.460749,56990.489821,58483.640654,60015.912039,61588.328935
1,Kilome,28132,28869.0584,29625.427730,30401.613937,31198.136222,32015.527391,32854.334208,33715.117765,34598.453850,35504.933341,36435.162594,37389.763854,38369.375667,39374.653310
2,Kaiti,28455,29200.5210,29965.574650,30750.672706,31556.340331,32383.116448,33231.554099,34102.220816,34995.699001,35912.586315,36853.496077,37819.057674,38809.916985,39826.736810
3,Makueni,50521,51844.6502,53202.980035,54596.898112,56027.336843,57495.253068,59001.628698,60547.471370,62133.815120,63761.721076,65432.278169,67146.603857,68905.844878,70711.178013
4,Kibwezi West,50889,52222.2918,53590.515845,54994.587360,56435.445549,57914.054223,59431.402443,60988.505187,62586.404023,64226.167808,65908.893405,67635.706412,69407.761920,71226.245283
5,Kibwezi East,34267,35164.7954,36086.113039,37031.569201,38001.796314,38997.443378,40019.176394,41067.678816,42143.652001,43247.815683,44380.908454,45543.688255,46736.932888,47961.440529


In [20]:
def woodlots_per_households(data:pd.DataFrame)->pd.DataFrame:
    df2 = data[['Sub-County','Household’s access to land (ha)','Average fraction of a smallholder farm under trees ','Average proportion/percentage of smallholder farms under trees)']]
    df2['Mean on-farm forests/woodlots (ha) per household'] = pd.to_numeric(df2['Household’s access to land (ha)'])*pd.to_numeric(df2['Average proportion/percentage of smallholder farms under trees)'])
    return df2



In [21]:
woodlots_per_households(df)

,Sub-County,Household’s access to land (ha),Average fraction of a smallholder farm under trees,Average proportion/percentage of smallholder farms under trees),Mean on-farm forests/woodlots (ha) per household
1,Mbooni,1.58,0.27,0.27,0.4266
2,Kilome,1.58,0.27,0.01,0.0158
3,Kaiti,1.58,0.45,0.87,1.3746
4,Makueni,1.58,0.1,0.13,0.2054
5,Kibwezi West,1.58,0.25,0.24,0.3792
6,Kibwezi East,1.58,0.25,0.05,0.0790


In [19]:
def expected_areas_under_trees(data:pd.DataFrame) ->pd.DataFrame:
    """
      NB: Accounting for homestead areas hived off from potential forest plantation area,
        assumed equal to 0.125 acre or 0.05 ha, as from 2023 going to the future
    """
    

1    1.58
2    1.58
3    1.58
4    1.58
5    1.58
6    1.58
Name: Household’s access to land (ha), dtype: float64

In [24]:
woods = woodlots_per_households(df)
house_holds = Households_accounting(df)

In [32]:
for i in house_holds.iterrows():
    print(i[10])

IndexError: tuple index out of range

In [29]:
(1,2,3)[1]

2